In [1]:
import requests
import base64 
import datetime
from urllib.parse import urlencode
import pprint

In [2]:
client_id = 'client id'
client_secret = 'client secret'

In [7]:
class SpotifyAPI():
    client_id = None
    client_secret = None
    access_token = None
    access_token_expires_in = datetime.datetime.now()
    access_token_did_expire = False
    token_url = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id , client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        client_id = self.client_id
        client_secret = self.client_secret
        if client_id == None or client_secret == None:
            raise Exception("You must set client id and client secret")
        client_creds = f"{self.client_id}:{self.client_secret}"
        client_creds_base64 = base64.b64encode(client_creds.encode())
        return client_creds_base64
    
    def get_token_header(self):
        client_creds_base64 = self.get_client_credentials()
        token_header = {
        "Authorization": f"Basic {client_creds_base64.decode()}"  # <base64 encoded client_id:client_secret>
        }
        return token_header
          
    def get_token_data(self):
        return {
                "grant_type":"client_credentials"
                }

    def perform_auth(self):
        token_data = self.get_token_data()
        token_header = self.get_token_header()
        r = requests.post(self.token_url, data = token_data, headers =token_header)
        if r.status_code not in range(200,299):
#             print(r.status_code)
            return False
        
        data = r.json()
        access_token = data['access_token']
        expires_in = data['expires_in']
        now = datetime.datetime.now()
        expires = now + datetime.timedelta(seconds = expires_in) # to get the time of expiration 
        self.access_token = access_token
        self.access_token_expires_in = expires
        self.access_token_did_expire = expires < now # token expired = if time of expiration is less than current time 
        return True
         
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires_in
        now = datetime.datetime.now()
        if now > expires:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token()
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
               "Authorization" : f"Bearer {access_token}"     
        }
        return headers
    
    def get_resource(self ,lookup_id , resource_type="artist" , version="v1" ):
        headers = self.get_resource_header()
        lookup_url = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        r = requests.get(lookup_url , headers= headers)
        if r.status_code not in range(200,299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id , resource_type="albums")
        
    def get_artist(self, _id):
        return self.get_resource(_id , resource_type="artists") 
        
    def search(self, query="Time", search_type="track"):
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({ "q": query, "type": search_type.lower() })
        lookup_url = f"{endpoint}{data}"
        r = requests.get(f"https://api.spotify.com/v1/search?{data}", headers = headers)
        pprint.pprint(r.json())
             

In [8]:
spotify = SpotifyAPI(client_id , client_secret)
# spotify.perform_auth()

In [9]:
# spotify.search(query="tania",search_type="Artist") 
spotify.get_artist("4rfeqiQmvXlc6LYxsgrXY7")
# spotify.get_album("1M4anG49aEs4YimBdj96Oy")

{'external_urls': {'spotify': 'https://open.spotify.com/artist/4rfeqiQmvXlc6LYxsgrXY7'},
 'followers': {'href': None, 'total': 9},
 'genres': [],
 'href': 'https://api.spotify.com/v1/artists/4rfeqiQmvXlc6LYxsgrXY7',
 'id': '4rfeqiQmvXlc6LYxsgrXY7',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab67616d0000b2734b17d92e59bf8b0953397803',
   'width': 640},
  {'height': 300,
   'url': 'https://i.scdn.co/image/ab67616d00001e024b17d92e59bf8b0953397803',
   'width': 300},
  {'height': 64,
   'url': 'https://i.scdn.co/image/ab67616d000048514b17d92e59bf8b0953397803',
   'width': 64}],
 'name': 'Tania Barron',
 'popularity': 39,
 'type': 'artist',
 'uri': 'spotify:artist:4rfeqiQmvXlc6LYxsgrXY7'}

In [10]:
# spotify.search(query="Arrival", type="album")
spotify.search("A lannister always pays his debts", search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=A+lannister+always+pays+his+debts&type=track&offset=0&limit=20',
            'items': [{'album': {'album_type': 'album',
                                 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1hCkSJcXREhrodeIHQdav8'},
                                              'href': 'https://api.spotify.com/v1/artists/1hCkSJcXREhrodeIHQdav8',
                                              'id': '1hCkSJcXREhrodeIHQdav8',
                                              'name': 'Ramin Djawadi',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:1hCkSJcXREhrodeIHQdav8'}],
                                 'available_markets': ['AD',
                                                       'AE',
                                                       'AR',
                                                       'AT',
                

                                             'BE',
                                             'BG',
                                             'BH',
                                             'BO',
                                             'BR',
                                             'CA',
                                             'CH',
                                             'CL',
                                             'CO',
                                             'CR',
                                             'CY',
                                             'CZ',
                                             'DE',
                                             'DK',
                                             'DO',
                                             'DZ',
                                             'EC',
                                             'EE',
                                             'EG',
                               

                                             'width': 640},
                                            {'height': 300,
                                             'url': 'https://i.scdn.co/image/ab67616d00001e02ffb7bc17d48213837323a016',
                                             'width': 300},
                                            {'height': 64,
                                             'url': 'https://i.scdn.co/image/ab67616d00004851ffb7bc17d48213837323a016',
                                             'width': 64}],
                                 'name': 'Music of Game of Thrones',
                                 'release_date': '2017-02-24',
                                 'release_date_precision': 'day',
                                 'total_tracks': 21,
                                 'type': 'album',
                                 'uri': 'spotify:album:0yfBIEEupVIj1cPS3yjuub'},
                       'artists': [{'external_urls': {'spotify': 'https://open.

                                              'name': "Divert'in Brass",
                                              'type': 'artist',
                                              'uri': 'spotify:artist:3flPrApBV2o2uxRawaUt4w'},
                                             {'external_urls': {'spotify': 'https://open.spotify.com/artist/6EscOBut2EiqD1twYzcihy'},
                                              'href': 'https://api.spotify.com/v1/artists/6EscOBut2EiqD1twYzcihy',
                                              'id': '6EscOBut2EiqD1twYzcihy',
                                              'name': 'Etienne Crausaz',
                                              'type': 'artist',
                                              'uri': 'spotify:artist:6EscOBut2EiqD1twYzcihy'}],
                                 'available_markets': ['AD',
                                                       'AE',
                                                       'AR',
                         